In [12]:
import pandas as pd
from sklearn.datasets import load_iris
import duckdb
import json
from typing import Optional, List, Dict, Any
from datetime import datetime, timedelta

def generate_table_schema(
    conn: duckdb.DuckDBPyConnection, 
    table_name: str, 
    list_all_values_columns: Optional[List[str]] = None, 
    skip_columns: Optional[List[str]] = None, 
    table_notes: Optional[str] = None,
    enable_filter: bool = True,
    filter_column: Optional[str] = None,
    filter_days_ago: int = 2
) -> Dict[str, Any]:
    """
    Generate schema context for a table in DuckDB.
    
    Args:
        conn: DuckDB connection
        table_name: Name of the table
        list_all_values_columns: List of columns to include all_possible_values for
        skip_columns: List of columns to skip entirely
        table_notes: Optional notes about the table
        enable_filter: Whether to apply date filtering
        filter_column: Column to filter on (defaults to 'dt' if exists)
        filter_days_ago: Number of days ago to filter by (default: 2)
    
    Returns:
        Dictionary with table schema context
    """
    if list_all_values_columns is None:
        list_all_values_columns = []
    if skip_columns is None:
        skip_columns = []
    
    # Get table schema as DataFrame
    schema_df = conn.execute(f"DESCRIBE {table_name}").df()
    column_names = schema_df['column_name'].tolist()
    
    # Determine filter settings and build base query
    actual_filter_column = None
    if enable_filter:
        if filter_column:
            actual_filter_column = filter_column
        elif 'dt' in column_names:
            actual_filter_column = 'dt'
    
    base_query = f"SELECT * FROM {table_name}"
    if actual_filter_column:
        filter_date = (datetime.now() - timedelta(days=filter_days_ago)).strftime('%Y-%m-%d')
        base_query += f" WHERE {actual_filter_column} = '{filter_date}'"
    
    # Get sample row as DataFrame
    sample_df = conn.execute(f"{base_query} LIMIT 1").df()
    
    schema = {
        table_name: {
            "columns": {}
        }
    }
    
    if table_notes:
        schema[table_name]["table_notes"] = table_notes
    
    for _, row in schema_df.iterrows():
        col_name = row['column_name']
        col_type = row['column_type']
        
        if col_name in skip_columns:
            continue
            
        col_info = {"type": col_type}
        
        # Add all_possible_values for specified columns
        if col_name in list_all_values_columns:
            # Replace SELECT * with SELECT DISTINCT col_name in base_query
            values_query = base_query.replace("SELECT *", f"SELECT DISTINCT {col_name}") + f" ORDER BY {col_name}"
            values_df = conn.execute(values_query).df()
            col_info["all_possible_values"] = values_df[col_name].tolist()
        # Add sample_value for non-numeric types by default (but not if all_possible_values exists)
        elif col_type not in ["DOUBLE", "BIGINT"]:
            if len(sample_df) > 0:  # Check if sample_df has data
                col_info["sample_value"] = sample_df[col_name].iloc[0]
        
        schema[table_name]["columns"][col_name] = col_info
    
    return schema

# Load the Iris dataset and create DuckDB table
iris = load_iris()
df = pd.DataFrame(iris.data, columns=iris.feature_names)
df['target'] = iris.target
df['target_name'] = df['target'].map({0: 'setosa', 1: 'versicolor', 2: 'virginica'})

conn = duckdb.connect('iris_dataset.db')
conn.execute("CREATE OR REPLACE TABLE iris AS SELECT * FROM df")

# Test the function
schema = generate_table_schema(
    conn, 
    "iris",
    list_all_values_columns=["target", "target_name"],
    skip_columns=[],
    table_notes="Iris flower dataset with measurements and species classification",
    enable_filter=False  # Disable filter since iris doesn't have dt column
)

print(json.dumps(schema, indent=2))

{
  "iris": {
    "columns": {
      "sepal length (cm)": {
        "type": "DOUBLE"
      },
      "sepal width (cm)": {
        "type": "DOUBLE"
      },
      "petal length (cm)": {
        "type": "DOUBLE"
      },
      "petal width (cm)": {
        "type": "DOUBLE"
      },
      "target": {
        "type": "BIGINT",
        "all_possible_values": [
          0,
          1,
          2
        ]
      },
      "target_name": {
        "type": "VARCHAR",
        "all_possible_values": [
          "setosa",
          "versicolor",
          "virginica"
        ]
      }
    },
    "table_notes": "Iris flower dataset with measurements and species classification"
  }
}
